In [14]:
import numpy as np
import konlpy
from konlpy.tag import Twitter
import os
from os import listdir
from collections import Counter

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

In [15]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [16]:
device

device(type='cuda', index=0)

# Hyperparameter

In [46]:
TEXT_DATA_DIR = './newsData'
NUM_CATEGORY = 8
BATCH_SIZE=10
EPOCHS=12
SPLIT_RATE=0.8
NUM_TEST_PER_DIRECTORY = 160

In [26]:
# unknown token 추가하여 초기화
word2idx = {'<unk>':0}
idx2word = {0:'<unk>'}
vocab=Counter()
vocab.update(['<unk>'])

# 데이터 전처리

In [19]:
def Extract_word(directory):
    print("%s 작업중"%directory)
    count = 1
    for filename in listdir(directory):
        # train 파일로만 단어장을 만든다.
        if count > NUM_TEST_PER_DIRECTORY:
            break
        path = directory+'/'+filename
        
        file = open(path,'r',encoding='utf-8')
        doc = file.read()
        file.close()
        
        twit = Twitter()
        doc = twit.pos(doc)
        tag=['Noun','Foreign','Alpha','Number']        
        exception=['\n\n','\t','‘','”',',','ㆍ','·','…','“','’','【','】','‧','◀','▶','\xa0','∼',"”…","ㆍ‘",
            '것','했',"있다","등","기자","또","며","그"]    
        for pair in doc:
            if pair[1] in tag and pair[0] not in exception:
                vocab.update([pair[0]])       
        count += 1

In [20]:
print(vocab)

Counter({'<unk>': 1})


In [37]:
# vocab 만들기, 이미 정의된 vocab이 있는 경우 생략한다.
def Make_vocab():
    if os.path.exists("vocab.txt"):
        print("이미 정의된 vocab을 불러옵니다.")
        file=open('vocab.txt','r',encoding='utf-8')
        doc=file.read()
        doc = doc.split('\n')
        file.close()
        return doc
    else: 
        for i in range(NUM_CATEGORY):
            Extract_word(TEXT_DATA_DIR+'/'+str(i))
        # vocab 저장
        data = '\n'.join(tokens)
        file=open('vocab.txt','w',encoding='utf-8')
        file.write(data)
        file.close()
        print('단어 수 :',len(vocab))
        print('빈출 상위 50 단어 :')
        print(vocab.most_common(50))
        return vocab

In [38]:
vocab = Make_vocab()

이미 정의된 vocab을 불러옵니다.


In [45]:
print(len(vocab))

15068


In [40]:
# i+1 을 한 이유는 <unk> 이 이미 하나 들어가 있기 때문
for i,k in enumerate(vocab):
    word2idx[k]=i+1
    idx2word[i+1]=k

다음에 할 일

데이터 
- 숫자면 다 숫자 토큰 하나로 변경하기(vocab에 숫자 토큰 추가)
- 단어장에 없던 단어가 나오면 <unk> 하나로 합치기
- word embedding 적용
- 모델 세우기(Attention Is All you need 논문 참고)
- 

In [ ]:
x_train=[]
t_train=[]
x_test=[]
t_test=[]

# 폴더 개수 8개
for i in range(NUM_CATEGORY):
    path = TEXT_DATA_DIR+'/'+str(i)
    print("%s folder 작업중..."%i)
    count = 0
    for filename in sorted(os.listdir(path)):
        filename=path+'/'+filename
        temp=Load_doc(filename)
        temp=Clean_doc(temp,vocab)
        temp=Word_To_Index(temp)
        # one hot encoding
        tempT=np.zeros(8)
        tempT[i]=1
        if count<160:
            x_train.append(temp)
            t_train.append(tempT)
        else :
            x_test.append(temp)
            t_test.append(tempT)
        count += 1
x_train=np.array(x_train)
x_test=np.array(x_test)
t_train=np.array(t_train)
t_test=np.array(t_test)
print(len(x_train))
print(len(t_train))
print(len(x_test))
print(len(t_test))

In [ ]:
# building Hierachical Attention Network
class Attention(nn.Module):
    def __init__(self,attention_size):
        super(Attention, self).__init__()
        self.attention = new_parameter(attention_size,1)
        
    def forward(self, x_in):
        # after this, we have (batch, dim1) with a diff weight per each cell
        attention_score = torch.matmul(x_in, self.attention).squeeze()
        attention_score = F.softmax(attention_score).view(x_in.size(0),x_in.size(1),1)